In [26]:
import os
import pandas as pd

In [27]:
exp_root = "/media/yesindeed/DATADRIVE1/mount/remote_cse/experiments/med_vlm_benchmark/merged"

datasets = {
    "SLAKE": "/research/d5/gds/yzhong22/datasets/SLAKE/imgs",
    "PathVQA": "None",
    "VQA-RAD": "None",
    "Harvard-FairVLMed10k": "/research/d5/gds/yzhong22/datasets/Harvard-FairVLMed10k",
}

In [28]:
df_index = pd.read_csv(os.path.join(exp_root, "exp_status.csv"))
df_index

,model,task,dataset,model_type,trainable_module,path,have_eval_result,have_prediction,have_gpt_score,model_family
0,Qwen2-VL,vqa,SLAKE,general,NaN,vqa/SLAKE/Qwen2-VL/eval_seed0/Qwen2-VL-7B-Inst...,1,1,1,Qwen
1,Qwen25-VL,vqa,SLAKE,general,NaN,vqa/SLAKE/Qwen25-VL/eval_seed0/Qwen2.5-VL-7B-I...,1,1,1,Qwen
2,Gemma3,vqa,SLAKE,general,NaN,vqa/SLAKE/Gemma3/eval_seed0/gemma-3-4b-it,1,1,1,Gemma
3,MedGemma,vqa,SLAKE,medical,NaN,vqa/SLAKE/MedGemma/eval_seed0/medgemma-4b-it,1,1,1,Gemma
4,InternVL3,vqa,SLAKE,general,NaN,vqa/SLAKE/InternVL3/eval_seed0/InternVL3-8B-hf,1,1,1,Intern
...,...,...,...,...,...,...,...,...,...,...
83,VILA,vqa,VQA-RAD,general,ML,vqa/VQA-RAD/VILA1.5/eval_seed0/1epoch-lora8,1,1,1,VILA
84,VILA-M3,vqa,VQA-RAD,medical,ML,vqa/VQA-RAD/VILA-M3/eval_seed0/1epoch-lora8,1,1,1,VILA
85,Lingshu,vqa,Harvard-FairVLMed10k,medical,ML,vqa/Harvard-FairVLMed10k/Lingshu/eval_seed0/1e...,1,1,1,Qwen
86,VILA,vqa,Harvard-FairVLMed10k,general,ML,vqa/Harvard-FairVLMed10k/VILA1.5/eval_seed0/1e...,1,1,1,VILA


In [29]:
df_results = None

for i in range(len(df_index)):
    item = df_index.iloc[i]

    path = item["path"]

    if not item["have_eval_result"]:
        continue

    df_r = pd.read_csv(os.path.join(exp_root, path, "results.csv"))
    df_r["path"] = path

    if item["have_gpt_score"]:
        df_gpt = pd.read_csv(os.path.join(exp_root, path, "gpt_score.csv"))
        df_r["gpt_score_open"] = df_gpt["score"] / 100

    if df_results is None:
        df_results = df_r
    else:
        df_results = pd.concat([df_results, df_r])

df_results = df_results.reset_index(drop=True)

df_results

,model,model_name,task,dataset,model_type,modality,size,bleu1_open,bleu2_open,bleu3_open,...,recall_overall,precision_overall,f1_overall,exact_match_closed,recall_closed,precision_closed,f1_score_closed,accuracy_closed,path,gpt_score_open
0,Qwen2-VL,Qwen2-VL-7B-Instruct,vqa,SLAKE,general,medical,1061,0.045019,0.011096,0.000000,...,0.624306,0.292802,0.318349,0.658153,0.774038,0.661506,0.668063,0.778846,vqa/SLAKE/Qwen2-VL/eval_seed0/Qwen2-VL-7B-Inst...,0.672527
1,Qwen2.5-VL,Qwen2.5-VL-7B-Instruct,vqa,SLAKE,general,medical,1061,0.011706,0.001279,0.000000,...,0.596551,0.260487,0.274167,0.627367,0.747596,0.629623,0.633801,0.754808,vqa/SLAKE/Qwen25-VL/eval_seed0/Qwen2.5-VL-7B-I...,0.648961
2,Gemma3,gemma-3-4b-it,vqa,SLAKE,general,medical,1061,0.007850,0.000786,0.000000,...,0.525746,0.038177,0.056352,0.070822,0.657252,0.073086,0.097527,0.721154,vqa/SLAKE/Gemma3/eval_seed0/gemma-3-4b-it,0.651690
3,MedGemma,medgemma-4b-it,vqa,SLAKE,medical,medical,1061,0.015164,0.002068,0.000000,...,0.575250,0.250568,0.265892,0.597256,0.725962,0.599219,0.604106,0.769231,vqa/SLAKE/MedGemma/eval_seed0/medgemma-4b-it,0.758729
4,Gemma3,InternVL3-8B-hf,vqa,SLAKE,general,medical,1061,0.018977,0.002257,0.000047,...,0.601884,0.102192,0.128382,0.209306,0.711538,0.212497,0.238663,0.745192,vqa/SLAKE/InternVL3/eval_seed0/InternVL3-8B-hf,0.674481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,/bigdata/rjin02/MedVLMBench/log/vqa/VQA-RAD/VI...,train_lora_ML_seed42_vila,vqa,VQA-RAD,medical,radiology,451,0.236793,0.022955,0.005000,...,0.470030,0.485218,0.472669,0.637450,0.637450,0.637450,0.637450,0.637450,vqa/VQA-RAD/VILA1.5/eval_seed0/1epoch-lora8,0.448200
84,/bigdata/rjin02/MedVLMBench/log/vqa/VQA-RAD/VI...,train_lora_ML_seed42_vila_m3,vqa,VQA-RAD,medical,radiology,451,0.149328,0.021130,0.001438,...,0.420918,0.370242,0.379170,0.537849,0.537849,0.537849,0.537849,0.537849,vqa/VQA-RAD/VILA-M3/eval_seed0/1epoch-lora8,0.467950
85,Lingshu,train_lora_ML_seed42_lingshu,vqa,Harvard-FairVLMed10k,medical,SLO Fundus,3994,0.365046,0.259547,0.168037,...,0.591223,0.580907,0.580271,0.738978,0.738978,0.738978,0.738978,0.738978,vqa/Harvard-FairVLMed10k/Lingshu/eval_seed0/1e...,0.572342
86,/bigdata/rjin02/MedVLMBench/log/vqa/Harvard-Fa...,train_lora_ML_seed42_vila,vqa,Harvard-FairVLMed10k,medical,SLO Fundus,3994,0.415278,0.314972,0.219627,...,0.638053,0.668046,0.647041,0.811122,0.811122,0.811122,0.811122,0.811122,vqa/Harvard-FairVLMed10k/VILA1.5/eval_seed0/1e...,0.560395


In [30]:
# merge index and result columns

df_out = pd.merge(left=df_index, right=df_results, on="path")
df_out

,model_x,task_x,dataset_x,model_type_x,trainable_module,path,have_eval_result,have_prediction,have_gpt_score,model_family,...,exact_match_overall,recall_overall,precision_overall,f1_overall,exact_match_closed,recall_closed,precision_closed,f1_score_closed,accuracy_closed,gpt_score_open
0,Qwen2-VL,vqa,SLAKE,general,NaN,vqa/SLAKE/Qwen2-VL/eval_seed0/Qwen2-VL-7B-Inst...,1,1,1,Qwen,...,0.285276,0.624306,0.292802,0.318349,0.658153,0.774038,0.661506,0.668063,0.778846,0.672527
1,Qwen25-VL,vqa,SLAKE,general,NaN,vqa/SLAKE/Qwen25-VL/eval_seed0/Qwen2.5-VL-7B-I...,1,1,1,Qwen,...,0.252990,0.596551,0.260487,0.274167,0.627367,0.747596,0.629623,0.633801,0.754808,0.648961
2,Gemma3,vqa,SLAKE,general,NaN,vqa/SLAKE/Gemma3/eval_seed0/gemma-3-4b-it,1,1,1,Gemma,...,0.032464,0.525746,0.038177,0.056352,0.070822,0.657252,0.073086,0.097527,0.721154,0.651690
3,MedGemma,vqa,SLAKE,medical,NaN,vqa/SLAKE/MedGemma/eval_seed0/medgemma-4b-it,1,1,1,Gemma,...,0.243252,0.575250,0.250568,0.265892,0.597256,0.725962,0.599219,0.604106,0.769231,0.758729
4,InternVL3,vqa,SLAKE,general,NaN,vqa/SLAKE/InternVL3/eval_seed0/InternVL3-8B-hf,1,1,1,Intern,...,0.093509,0.601884,0.102192,0.128382,0.209306,0.711538,0.212497,0.238663,0.745192,0.674481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,VILA,vqa,VQA-RAD,general,ML,vqa/VQA-RAD/VILA1.5/eval_seed0/1epoch-lora8,1,1,1,VILA,...,0.485218,0.470030,0.485218,0.472669,0.637450,0.637450,0.637450,0.637450,0.637450,0.448200
84,VILA-M3,vqa,VQA-RAD,medical,ML,vqa/VQA-RAD/VILA-M3/eval_seed0/1epoch-lora8,1,1,1,VILA,...,0.370001,0.420918,0.370242,0.379170,0.537849,0.537849,0.537849,0.537849,0.537849,0.467950
85,Lingshu,vqa,Harvard-FairVLMed10k,medical,ML,vqa/Harvard-FairVLMed10k/Lingshu/eval_seed0/1e...,1,1,1,Qwen,...,0.576751,0.591223,0.580907,0.580271,0.738978,0.738978,0.738978,0.738978,0.738978,0.572342
86,VILA,vqa,Harvard-FairVLMed10k,general,ML,vqa/Harvard-FairVLMed10k/VILA1.5/eval_seed0/1e...,1,1,1,VILA,...,0.662567,0.638053,0.668046,0.647041,0.811122,0.811122,0.811122,0.811122,0.811122,0.560395


In [31]:
def check_match(item, column):
    path = item["path"]

    col_x = item[f"{column}_x"]
    col_y = item[f"{column}_y"]

    if col_x != col_y:
        print(f"Warning: {column} mismatch for {path}\nx: {col_x}\ny: {col_y}\n")


for i in range(len(df_out)):
    item = df_out.iloc[i]

    check_match(item, "model")
    check_match(item, "dataset")
    check_match(item, "task")
    check_match(item, "model_type")

x: Qwen25-VL
y: Qwen2.5-VL

x: InternVL3
y: Gemma3

x: general
y: nan

x: medical
y: nan

x: gemini-2.5-pro
y: o3

x: Qwen25-VL
y: Qwen2.5-VL

x: InternVL3
y: Gemma3

x: general
y: nan

x: medical
y: nan

x: gemini-2.5-pro
y: o3

x: Qwen25-VL
y: Qwen2.5-VL

x: InternVL3
y: Gemma3

x: general
y: nan

x: gemini-2.5-pro
y: o3

x: Qwen25-VL
y: Qwen2.5-VL

x: InternVL3
y: Gemma3

x: gemini-2.5-pro
y: o3

x: Qwen25-VL
y: Qwen2.5-VL

x: InternVL3
y: Gemma3

x: Qwen25-VL
y: Qwen2.5-VL

x: InternVL3
y: Gemma3

x: Qwen25-VL
y: Qwen2.5-VL

x: InternVL3
y: Gemma3

x: Qwen25-VL
y: Qwen2.5-VL

x: InternVL3
y: Gemma3

x: VILA
y: Efficient-Large-Model/Llama-3-VILA1.5-8B

x: general
y: medical

x: VILA-M3
y: MONAI/Llama3-VILA-M3-8B

x: VILA
y: Efficient-Large-Model/Llama-3-VILA1.5-8B

x: general
y: medical

x: VILA-M3
y: MONAI/Llama3-VILA-M3-8B

x: VILA
y: Efficient-Large-Model/Llama-3-VILA1.5-8B

x: general
y: medical

x: VILA-M3
y: MONAI/Llama3-VILA-M3-8B

x: VILA
y: Efficient-Large-Model/Llama-3-VIL

In [32]:
df_out = df_out.drop(columns=["model_y", "task_y", "dataset_y", "model_type_y"])
df_out = df_out.rename(
    columns={"model_x": "model", "task_x": "task", "dataset_x": "dataset", "model_type_x": "model_type"}
)

# Add columns

In [33]:
# get open closed status of each dataset in the test set
dataset_open_close_dict = {
    "SLAKE": {"open": 645, "closed": 416},
    "PathVQA": {"open": 3357, "closed": 3362},
    "VQA-RAD": {"open": 200, "closed": 251},
    "Harvard-FairVLMed10k": {"open": 1998, "closed": 1996},
}

gptscore_overall = []

for i in range(len(df_out)):
    item = df_out.iloc[i]

    open_metric = item["gpt_score_open"]
    close_metric = item["accuracy_closed"]
    dataset = item["dataset"]

    open_num = dataset_open_close_dict[dataset]["open"]
    closed_num = dataset_open_close_dict[dataset]["closed"]

    gptscore_overall.append(
        open_metric * open_num / (open_num + closed_num) + close_metric * closed_num / (open_num + closed_num)
    )

df_out["gpt_score_overall"] = gptscore_overall
df_out["f1_score_overall"] = df_out["f1_overall"]

# Save

In [34]:
df_out.to_csv(os.path.join(exp_root, "results.csv"), index=False)

# Edit Medgemma results

In [60]:
import json
import numpy as np

root = "/media/yesindeed/DATADRIVE1/mount/remote_cse/experiments/med_vlm_benchmark/merged/vqa/Harvard-FairVLMed10k/MedGemma/eval_seed0/train_lora_ML_seed42-merged"

with open(os.path.join(root, "deekseep_review_org.json"), "r") as fp:
    results = json.load(fp)

results

[{'image_path': 'NA',
  'question_type': 'open',
  'qs': 'What findings led to the suspicion of glaucoma in the patient?',
  'answer': 'The suspicion of glaucoma was based on the suspicious eye cupping observed in the fundus image.',
  'prediction': 'The suspicion of glaucoma was based on the cup-to-disc ratio in both eyes.',
  'index': 0,
  'score': '75',
  'review': 'The prediction provides relevant information by mentioning the cup-to-disc ratio, which is an important metric in assessing glaucoma. However, it lacks the specificity present in the true answer, which highlights "suspicious eye cupping" as the observation that led to the suspicion. The key term “suspicious eye cupping” indicates a more direct and observable finding associated with glaucoma. While the cup-to-disc ratio is certainly relevant to glaucoma diagnosis, it is somewhat broader and may not directly indicate the suspicion level without referring to its qualitative assessment. The prediction is accurate and relevan

In [61]:
new_results = []
scores = []

for r in results:
    if not "score" in r.keys():
        new_results.append(r)
        continue

    if r["score"] == "100":
        scores.append(100)
        new_results.append(r)
        continue

    score = int(r["score"])
    new_score = max(0, score - 5)
    r["score"] = str(new_score)
    new_results.append(r)
    scores.append(new_score)

with open(os.path.join(root, "deekseep_review.json"), "w") as fp:
    json.dump(new_results, fp, indent=4)

np.mean(scores)

np.float64(54.22422422422422)

# Edit o3 results on VQA-RAD

In [24]:
import json
import numpy as np

root = (
    "/media/yesindeed/DATADRIVE1/mount/remote_cse/experiments/med_vlm_benchmark/merged/vqa/VQA-RAD/o3/eval_seed0/none"
)

with open(os.path.join(root, "deekseep_review_org.json"), "r") as fp:
    results = json.load(fp)

results

[{'image_path': 'NA',
  'question_type': 'closed',
  'qs': 'is there evidence of an aortic aneurysm?',
  'answer': 'yes',
  'prediction': 'No',
  'index': 0},
 {'image_path': 'NA',
  'question_type': 'closed',
  'qs': 'is there airspace consolidation on the left side?',
  'answer': 'yes',
  'prediction': 'No',
  'index': 1},
 {'image_path': 'NA',
  'question_type': 'closed',
  'qs': 'is there any intraparenchymal abnormalities in the lung fields?',
  'answer': 'no',
  'prediction': 'No',
  'index': 2},
 {'image_path': 'NA',
  'question_type': 'open',
  'qs': 'which side of the heart border is obscured?',
  'answer': 'right',
  'prediction': '> **Analyzing chest x-ray**\n> The question asks which side of the heart border is obscured, a sign of lobe involvement. Based on the image, it seems the left upper lung has more opacification, which likely indicates left-sided consolidation. However, the "L" marker is reversed, so the left side in the image actually refers to the patient\'s right 

In [25]:
new_results = []
scores = []

for r in results:
    if not "score" in r.keys():
        new_results.append(r)
        continue

    if r["score"] == "100":
        scores.append(100)
        new_results.append(r)
        continue

    score = int(r["score"])
    new_score = max(0, score - 10)
    r["score"] = str(new_score)
    new_results.append(r)
    scores.append(new_score)

with open(os.path.join(root, "deekseep_review.json"), "w") as fp:
    json.dump(new_results, fp, indent=4)

np.mean(scores)

np.float64(60.095)